# Turing Machine
This notebook simulates a turing machine, reducing its basxic concepts to python structures.

In [1]:
import re

## Defining the turing machine and its operations

In [2]:
class TuringMachine:
    def __init__(self, tape="", blank_symbol=" "):
        self.tape = [blank_symbol] + list(tape) + [blank_symbol]
        self.head_position = 1
        self.blank_symbol = blank_symbol
        self.current_state = "start"
        self.halted = False
        self.transitions = {
            #Add
            ("start", "1"): ("1", "R", "start"),
            ("start", "+"): ("", "R", "Adding"),
            ("Adding", "1"): ("1", "R", "Adding"),
            #Subtract
            ("start", "-"): ("", "R", "SubtractingR"),
            ("SubtractingR", "1"): ("", "L", "SubtractingL"),
            ("SubtractingR", "") : ("", "R", "SubtractingR"),
            ("SubtractingL", "1"): ("", "R", "SubtractingR"),
            ("SubtractingL", "") : ("", "L", "SubtractingL"),
            ("SubtractingL", " ") : ("-", "R", "AddOne"),
            ("AddOne", "") : ("1", "R", "FinishedSubtracting"),
            # Multiplication
            ("start", "*") : ("*", "L", "FindStart"),
            ("FindStart", "1") : ("1", "L", "FindStart"),
            ("FindStart", " ") : (" ", "R", "MoveStart"),
            ("MoveStart", "1") : (" ", "R", "Find*"),
            ("Find*", "1") : ("1", "R", "Find*"),
            ("Find*", "*") : ("*", "R", "CreateX"),
            ("CreateX", "1") : ("X", "R", "Prepare"),
            ("Prepare", "1") : ("1", "R", "Prepare"),
            ("Prepare", " ") : ("", "R", "Duplicate"),
            ("Prepare", "") : ("", "R", "Duplicate"),
            ("Duplicate", "1") : ("1", "R", "Duplicate"),
            ("Duplicate", " ") : ("1", "L", "MoveBack"),
            ("MoveBack", "1") : ("1", "L", "MoveBack"),
            ("MoveBack", "") : ("", "L", "CheckX"),
            ("CheckX", "1") : ("1", "L", "MoveBack"),
            ("CheckX", "X") : ("1", "L", "Back*"),
            ("MoveBack", "X") : ("X", "R", "CreateX"),
            ("Back*", "X") : ("1", "L", "Back*"),
            ("Back*", "*") : ("*", "L", "CheckStart"),
            ("CheckStart", "1") : ("1", "L", "FindStart"),
            ("CheckStart", " ") : (" ", "R", "Delete"),
            ("Delete", "*") : ("", "R", "Delete"),
            ("Delete", "1") : ("", "R", "Delete"),
            ("Delete", "") : ("", "R", "FinishedMultiplying"),
            # Division
            ("start", "/") : ("/", "R", "CheckZero"),
            ("CheckZero", " ") : (" Undefined ", "L", "ERROR"),
            ("CheckZero", "1") : ("1", "R", "PrepareDiv"),
            ("PrepareDiv", "1") : ("1", "R", "PrepareDiv"),
            ("PrepareDiv", "X") : ("X", "R", "PrepareDiv"),
            ("PrepareDiv", " ") : ("", "L", "CreateXDiv"),
            ("PrepareDiv", "") : ("", "L", "CreateXDiv"),
            ("CreateXDiv", "X") : ("X", "L", "CreateXDiv"),
            ("CreateXDiv", "1") : ("X", "L", "Find/"),
            ("Find/", "1") : ("1", "L", "Find/"),
            ("Find/", "/") : ("/", "L", "FindStartDiv"),
            ("FindStartDiv", "1") : ("1", "L", "FindStartDiv"),
            ("FindStartDiv", " ") : (" ", "R", "MoveStartDiv"),
            ("MoveStartDiv", "1") : (" ", "R", "FindF/"),
            ("FindF/", "1") : ("1", "R", "FindF/"),
            ("FindF/", "/") : ("/", "R", "CheckXDiv"),
            ("CheckXDiv", "1") : ("1", "R", "PrepareDiv"),
            ("CheckXDiv", "X") : ("1", "R", "AddDiv"),
            ("AddDiv", "X") : ("1", "R", "AddDiv"),
            ("AddDiv", "1") : ("1", "R", "AddDiv"),
            ("AddDiv", "") : ("", "R", "AddDiv"),
            ("AddDiv", " ") : ("1", "L", "MoveBackDiv"),
            ("MoveBackDiv", "1") : ("1", "L", "MoveBackDiv"),
            ("MoveBackDiv", "") : ("", "L", "CreateXDiv"),
            ("MoveStartDiv", "/") : ("", "R", "Delete"),
            ("Delete", "1") : ("", "R", "Delete"),
            ("Delete", "X") : ("", "R", "Delete"),
            ("Delete", "") : ("", "R", "FinishedDividing"),
            # Modulus
            ("start", "%") : ("%", "R", "CheckZeroModulus"),
            ("CheckZeroModulus", " ") : (" Undefined ", "L", "ERROR"),
            ("CheckZeroModulus", "1") : ("1", "R", "PrepareMod"),
            ("PrepareMod", "1") : ("1", "R", "PrepareMod"),
            ("PrepareMod", "X") : ("X", "R", "PrepareMod"),
            ("PrepareMod", " ") : ("", "L", "CreateXMod"),
            ("PrepareMod", "") : ("", "L", "CreateXMod"),
            ("CreateXMod", "X") : ("X", "L", "CreateXMod"),
            ("CreateXMod", "1") : ("X", "L", "Find%"),
            ("Find%", "1") : ("1", "L", "Find%"),
            ("Find%", "%") : ("%", "L", "FindStartMod"),
            ("FindStartMod", "1") : ("1", "L", "FindStartMod"),
            ("FindStartMod", " ") : (" ", "R", "MoveStartMod"),
            ("MoveStartMod", "1") : (" ", "R", "FindF%"),
            ("FindF%", "1") : ("1", "R", "FindF%"),
            ("FindF%", "%") : ("%", "R", "CheckXMod"),
            ("CheckXMod", "1") : ("1", "R", "PrepareMod"),
            ("CheckXMod", "X") : ("1", "R", "AddMod"),
            ("AddMod", "X") : ("1", "R", "AddMod"),
            ("AddMod", "1") : ("1", "R", "AddMod"),
            ("AddMod", "") : ("", "R", "AddMod"),
            ("AddMod", " ") : ("1", "L", "MoveBackMod"),
            ("MoveBackMod", "1") : ("1", "L", "MoveBackMod"),
            ("MoveBackMod", "") : ("", "L", "CheckXMod"),
            ("MoveStartMod", "%") : ("", "R", "FindFirstX"),
            ("FindFirstX", "1") : ("", "R", "FindFirstX"),
            ("FindFirstX", "X") : ("", "R", "CountMod"),
            ("CountMod", "X") : ("1", "R", "CountMod"),
            ("CountMod", "1") : ("", "R", "CountMod"),
            ("CountMod", "") : ("", "R", "CountMod"),
            ("CountMod", " ") : (" ", "R", "FinishedModulus"),
        }

    
    def step(self):
        if self.halted:
            raise Exception("Machine is halted")
        current_symbol = self.tape[self.head_position]
        action = self.transitions.get((self.current_state, current_symbol))
        if action:
            symbol_to_write, move_direction, next_state = action
            self.tape[self.head_position] = symbol_to_write
            if move_direction == "R":
                self.head_position += 1
            elif move_direction == "L":
                self.head_position -= 1
            if self.head_position < 0:
                self.head_position = 0
                self.tape.insert(0, self.blank_symbol)
            elif self.head_position >= len(self.tape):
                self.tape.append(self.blank_symbol)
            self.current_state = next_state
        else:
            self.halted = True

    def run(self):
        while not self.halted:
            self.step()

    def add_transition(self, state, symbol, write_symbol, direction, next_state):
        self.transitions[(state, symbol)] = (write_symbol, direction, next_state)

    def __str__(self):
        tape_str = "".join(self.tape).rstrip(self.blank_symbol)
        return f"Tape: {tape_str}\nHead at: {self.head_position}\nState: {self.current_state}"


In [3]:
def to_unary(n):
    return "1" * n
def from_unary(s):
    return str(s[0]) + str(s.count("1"))
def convert_to_unary_expression(s):
    return re.sub(r"\d+", lambda m: to_unary(int(m.group(0))), s)

## Testing

In [5]:
expression = "2*10"
print("Expression:", convert_to_unary_expression(expression))
tm = TuringMachine(convert_to_unary_expression(expression)) 
tm.run()
print(tm)
print("Decimal Result: ",from_unary(tm.tape))

Expression: 11*1111111111
Tape:    11111111111111111111
Head at: 15
State: FinishedDividing
Decimal Result:   20
